<a href="https://colab.research.google.com/github/Dhrupu/CNN/blob/main/Bagged1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import cv2
import random
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model

In [ ]:
IMG_SIZE = (128, 128)

In [ ]:
# Load images and extract features
def load_images_and_labels(data_dir, classes):
    X, y = [], []

    for label, class_name in enumerate(classes):
        class_dir = os.path.join(data_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            try:
                img = cv2.imread(img_path)
                img = cv2.resize(img, IMG_SIZE)
                X.append(img)
                y.append(label)
            except:
                continue

    return np.array(X), np.array(y)

In [ ]:
train_dir = "/content/drive/MyDrive/guava dataset/GuavaDiseaseDataset/train"
val_dir = "/content/drive/MyDrive/guava dataset/GuavaDiseaseDataset/val"

In [ ]:
# Get class labels
classes = os.listdir(train_dir)

# Load train and validation data
X_train, y_train = load_images_and_labels(train_dir, classes)
X_val, y_val = load_images_and_labels(val_dir, classes)

# Normalize image data
X_train = X_train / 255.0
X_val = X_val / 255.0


In [ ]:
# Feature Extraction using VGG16 (Pretrained CNN)
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(128, 128, 3))
feature_extractor = Model(inputs=base_model.input, outputs=base_model.output)


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
# Extract features
X_train_features = feature_extractor.predict(X_train)
X_val_features = feature_extractor.predict(X_val)


83/83 ━━━━━━━━━━━━━━━━━━━━ 509s 6s/step
24/24 ━━━━━━━━━━━━━━━━━━━━ 136s 6s/step


In [ ]:
# Flatten features for Decision Trees
X_train_features = X_train_features.reshape(X_train_features.shape[0], -1)
X_val_features = X_val_features.reshape(X_val_features.shape[0], -1)


In [ ]:
# Train Bagged Trees (Random Forest)
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_features, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
# Predict on validation data
y_pred = rf_model.predict(X_val_features)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_val, y_pred)
print(f"Validation Accuracy: {accuracy:.4f}")
print(classification_report(y_val, y_pred, target_names=classes))


Validation Accuracy: 0.9272
               precision    recall  f1-score   support

    fruit_fly       0.93      0.89      0.91       262
healthy_guava       0.91      0.93      0.92       185
  Anthracnose       0.94      0.96      0.95       308

     accuracy                           0.93       755
    macro avg       0.92      0.93      0.93       755
 weighted avg       0.93      0.93      0.93       755

